In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from __future__ import division
from __future__ import print_function
from operator import itemgetter
from itertools import combinations
import time
import os

import tensorflow as tf
import numpy as np
import networkx as nx
import scipy.sparse as sp
import scipy.io as sio
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import pickle

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'      #Indra: device set to 0
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.disable_eager_execution()  # Indra: Disable eager execution for compat

np.random.seed(0)

# DATA LOADING

In [ ]:
#LOAD Disease Gene Adjacency Network
gene_phenes_path = '/content/drive/MyDrive/data_prioritization/genes_phenes.mat'
f = h5py.File(gene_phenes_path, 'r')
gene_network_adj = sp.csc_matrix((np.array(f['GeneGene_Hs']['data']),
    np.array(f['GeneGene_Hs']['ir']), np.array(f['GeneGene_Hs']['jc'])),
    shape=(12331,12331))
gene_network_adj = gene_network_adj.tocsr()

In [ ]:
def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [ ]:
def network_edge_threshold(network_adj, threshold):
    edge_tmp, edge_value, shape_tmp = sparse_to_tuple(network_adj)
    preserved_edge_index = np.where(edge_value>threshold)[0]
    preserved_network = sp.csr_matrix(
        (edge_value[preserved_edge_index],
        (edge_tmp[preserved_edge_index,0], edge_tmp[preserved_edge_index, 1])),
        shape=shape_tmp)
    return preserved_network

In [ ]:
disease_network_adj = sp.csc_matrix((np.array(f['PhenotypeSimilarities']['data']),
    np.array(f['PhenotypeSimilarities']['ir']), np.array(f['PhenotypeSimilarities']['jc'])),
    shape=(3215, 3215))
disease_network_adj = disease_network_adj.tocsr()  ## CONVERTED TO SPARSE ROW
# >0.2 values get preserved
disease_network_adj = network_edge_threshold(disease_network_adj, 0.2)

In [ ]:
## NOTE: Gene Disease Adj is 1, 0 matrix, while gene_gene (12331, 12331)
##        and disease-disease ( 3215, 3215) have edge level values

dg_ref = f['GenePhene'][0][0]
gene_disease_adj = sp.csc_matrix((np.array(f[dg_ref]['data']),
    np.array(f[dg_ref]['ir']), np.array(f[dg_ref]['jc'])),
    shape=(12331, 3215))
gene_disease_adj = gene_disease_adj.tocsr()

In [ ]:
## WHAT DATA is THIS ? 34 novel associtaions of value 1, 0
novel_associations_adj = sp.csc_matrix((np.array(f['NovelAssociations']['data']),
    np.array(f['NovelAssociations']['ir']), np.array(f['NovelAssociations']['jc'])),
    shape=(12331,3215))


In [ ]:
#Feature value of each Gene: 4536 features per gene
gene_feature_path = '/content/drive/MyDrive/data_prioritization/GeneFeatures.mat'
f_gene_feature = h5py.File(gene_feature_path,'r')
gene_feature_exp = np.array(f_gene_feature['GeneFeatures'])
gene_feature_exp = np.transpose(gene_feature_exp)
gene_network_exp = sp.csc_matrix(gene_feature_exp)

In [ ]:
# // TODO: Explore why 1 - 9 and not 0 - 9
## NOTE THIS DATA is different from gene_disease_adj

row_list = [3215, 1137, 744, 2503, 1143, 324, 1188, 4662, 1243]
gene_feature_list_other_spe = list()
for i in range(1,9):
    dg_ref = f['GenePhene'][i][0]
    disease_gene_adj_tmp = sp.csc_matrix((np.array(f[dg_ref]['data']),
        np.array(f[dg_ref]['ir']), np.array(f[dg_ref]['jc'])),
        shape=(12331, row_list[i]))
    gene_feature_list_other_spe.append(disease_gene_adj_tmp)

In [ ]:
disease_tfidf_path = '/content/drive/MyDrive/data_prioritization/clinicalfeatures_tfidf.mat'
f_disease_tfidf = h5py.File(disease_tfidf_path)
disease_tfidf = np.array(f_disease_tfidf['F'])
disease_tfidf = np.transpose(disease_tfidf)
disease_tfidf = sp.csc_matrix(disease_tfidf)

In [ ]:
dis_dis_adj_list= list()
dis_dis_adj_list.append(disease_network_adj)

# DATA PARAMS

In [ ]:
val_test_size = 0.1
n_genes = 12331
n_dis = 3215
n_dis_rel_types = len(dis_dis_adj_list)
gene_adj = gene_network_adj
gene_degrees = np.array(gene_adj.sum(axis=0)).squeeze()
gene_degrees.shape

In [ ]:
gene_dis_adj = gene_disease_adj
dis_gene_adj = gene_dis_adj.transpose(copy=True)
dis_degrees_list = [np.array(dis_adj.sum(axis=0)).squeeze() for dis_adj in dis_dis_adj_list]
dis_degrees_list[0].shape

In [ ]:
adj_mats_orig = {
    (0, 0): [gene_adj, gene_adj.transpose(copy=True)],
    (0, 1): [gene_dis_adj],
    (1, 0): [dis_gene_adj],
    (1, 1): dis_dis_adj_list + [x.transpose(copy=True) for x in dis_dis_adj_list],
}
degrees = {
    0: [gene_degrees, gene_degrees],
    1: dis_degrees_list + dis_degrees_list,
}

In [ ]:
# Gene Feature Exp - Feature vector  + other 8 species
gene_feat = sp.hstack(gene_feature_list_other_spe+[gene_feature_exp])
gene_nonzero_feat, gene_num_feat = gene_feat.shape
gene_feat = sparse_to_tuple(gene_feat.tocoo())

dis_feat = disease_tfidf
dis_nonzero_feat, dis_num_feat = dis_feat.shape
dis_feat = sparse_to_tuple(dis_feat.tocoo())


In [ ]:
num_feat = {
    0: gene_num_feat,
    1: dis_num_feat,
}
nonzero_feat = {
    0: gene_nonzero_feat,
    1: dis_nonzero_feat,
}
feat = {
    0: gene_feat,
    1: dis_feat,
}

In [ ]:
edge_type2dim = {k: [adj.shape for adj in adjs] for k, adjs in adj_mats_orig.items()}
# edge_type2decoder = {
#     (0, 0): 'bilinear',
#     (0, 1): 'bilinear',
#     (1, 0): 'bilinear',
#     (1, 1): 'bilinear',
# }

edge_type2decoder = {
    (0, 0): 'innerproduct',
    (0, 1): 'innerproduct',
    (1, 0): 'innerproduct',
    (1, 1): 'innerproduct',
}

edge_types = {k: len(v) for k, v in adj_mats_orig.items()}
num_edge_types = sum(edge_types.values())
print("Edge types:", "%d" % num_edge_types)

# INSIDE MAIN CALLING CODE

# FLAGS

By Indra: Converted all flags to variables

In [ ]:
neg_sample_size =  1  #Negative sample size
learning_rate =  0.01 # 'Initial learning rate
hidden1_units =  64 #'Number of units in hidden layer 1
hidden2_units = 32 #Number of units in hidden layer 2
weight_decay =  0.001  #Weight for L2 loss on embedding matrix
dropout = 0.1 # Dropout rate (1 - keep probability)
max_margin =  0.1 #Max margin parameter in hinge loss
batch_size = 512
bias = True #Bias term

In [ ]:
def construct_placeholders(edge_types):
    placeholders = {
        'batch': tf.compat.v1.placeholder(tf.int32, name='batch'),
        'batch_edge_type_idx': tf.compat.v1.placeholder(tf.int32, shape=(), name='batch_edge_type_idx'),
        'batch_row_edge_type': tf.compat.v1.placeholder(tf.int32, shape=(), name='batch_row_edge_type'),
        'batch_col_edge_type': tf.compat.v1.placeholder(tf.int32, shape=(), name='batch_col_edge_type'),
        'degrees': tf.compat.v1.placeholder(tf.int32),
        'dropout': tf.compat.v1.placeholder_with_default(0., shape=()),
        'neg_sample_indexes': tf.compat.v1.placeholder(tf.int32),
    }
    placeholders.update({
        'adj_mats_%d,%d,%d' % (i, j, k): tf.compat.v1.sparse_placeholder(tf.float32)
        for i, j in edge_types for k in range(edge_types[i,j])})
    placeholders.update({
        'feat_%d' % i: tf.compat.v1.sparse_placeholder(tf.float32)
        for i, _ in edge_types})
    return placeholders

In [ ]:
print("Defining placeholders")
placeholders = construct_placeholders(edge_types)

In [ ]:
from collections import defaultdict
epoch_losses_history = defaultdict(list,)
current_epoch_loss = [[] for _ in range(num_edge_types)]

# Defining Minibatches

In [ ]:
class EdgeMinibatchIterator(object):
    """ This minibatch iterator iterates over batches of sampled edges or
    random pairs of co-occuring edges.
    assoc -- numpy array with target edges
    placeholders -- tensorflow placeholders object
    batch_size -- size of the minibatches
    """
    def __init__(self, adj_mats, feat, edge_types, batch_size=100, val_test_size=0.01):
        self.adj_mats = adj_mats  #Cell 26
        self.feat = feat           #{ 0: [12331, 17480 feature matrix], 1: [3215, 16592 feature matrix] }
        self.edge_types = edge_types     #{(0, 0): 2, (0, 1): 1, (1, 0): 1, (1, 1): 2}
        self.batch_size = batch_size         #512
        self.val_test_size = val_test_size       #0.1
        self.num_edge_types = sum(self.edge_types.values())        #6

        self.iter = 0
        self.freebatch_edge_types= list(range(self.num_edge_types))            #{[0,1,2,3,4,5]}
        self.batch_num = [0]*self.num_edge_types        #[0,0,0,0,0,0]
        self.current_edge_type_idx = 0
        self.edge_type2idx = {}
        self.idx2edge_type = {}
        r = 0
        for i, j in self.edge_types:
            for k in range(self.edge_types[i,j]):
                self.edge_type2idx[i, j, k] = r          #{(0,0,0): 0, (0,0,1): 1 ...}
                self.idx2edge_type[r] = i, j, k          #{0: (0,0,0), 1: (0,0,1) ...}
                r += 1

        self.train_edges = {edge_type: [None]*n for edge_type, n in self.edge_types.items()} #{(0,0):[None, None], ..}
        self.val_edges = {edge_type: [None]*n for edge_type, n in self.edge_types.items()}
        self.test_edges = {edge_type: [None]*n for edge_type, n in self.edge_types.items()}
        self.test_edges_false = {edge_type: [None]*n for edge_type, n in self.edge_types.items()}
        self.val_edges_false = {edge_type: [None]*n for edge_type, n in self.edge_types.items()}

        # Function to build test and val sets with val_test_size positive links
        self.adj_train = {edge_type: [None]*n for edge_type, n in self.edge_types.items()} #{(0,0):[None, None], ..}
        for i, j in self.edge_types:
            for k in range(self.edge_types[i,j]):
                self.mask_test_edges((i, j), k)

    def mask_test_edges(self, edge_type, type_idx):
        edges_all, _, _ = sparse_to_tuple(self.adj_mats[edge_type][type_idx])  #Cell 26 convert to {(n*2: x,y),n*val,shape} Cell 40
        num_test = max(100, int(np.floor(edges_all.shape[0] * self.val_test_size)))  #edges_all are the coords in adj matrix
        num_val = max(100, int(np.floor(edges_all.shape[0] * self.val_test_size)))   #num_test is max 100 ??? WHYYYY??

        if edge_type not in [(0,1), (1, 0)]:   # DIS_DIS and GENE_GENE only 10 ???? WHYY??
            num_test = 10
            num_val = 10

        all_edge_idx = list(range(edges_all.shape[0]))  #edges_all.shape[0] - > Num of non_zero values in sparse [0,1 .. N]
        np.random.shuffle(all_edge_idx)

        val_edge_idx = all_edge_idx[:num_val]
        val_edges = edges_all[val_edge_idx]

        test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
        test_edges = edges_all[test_edge_idx]

        train_edges = np.delete(edges_all, np.hstack([test_edge_idx, val_edge_idx]), axis=0)

        #Checks if edges are part of edges_all, then adds to test_edges_false if not there, same length as test_edges
        test_edges_false = []
        while len(test_edges_false) < len(test_edges):
            idx_i = np.random.randint(0, self.adj_mats[edge_type][type_idx].shape[0])
            idx_j = np.random.randint(0, self.adj_mats[edge_type][type_idx].shape[1])
            if self._ismember([idx_i, idx_j], edges_all):
                continue
            if test_edges_false:
                if self._ismember([idx_i, idx_j], test_edges_false):
                    continue
            test_edges_false.append([idx_i, idx_j])

        val_edges_false = []
        while len(val_edges_false) < len(val_edges):
            idx_i = np.random.randint(0, self.adj_mats[edge_type][type_idx].shape[0])
            idx_j = np.random.randint(0, self.adj_mats[edge_type][type_idx].shape[1])
            if self._ismember([idx_i, idx_j], edges_all):
                continue
            if val_edges_false:
                if self._ismember([idx_i, idx_j], val_edges_false):
                    continue
            val_edges_false.append([idx_i, idx_j])

        #ALL _edges and _edges_false are arrays of coordinates [n*[x,y]]
        # Re-build adj matrices
        data = np.ones(train_edges.shape[0])
        adj_train = sp.csr_matrix(
            (data, (train_edges[:, 0], train_edges[:, 1])),
            shape=self.adj_mats[edge_type][type_idx].shape)        #adj_train are now just 1's
        self.adj_train[edge_type][type_idx] = self.preprocess_graph(adj_train)

        self.train_edges[edge_type][type_idx] = train_edges
        self.val_edges[edge_type][type_idx] = val_edges
        self.val_edges_false[edge_type][type_idx] = np.array(val_edges_false)
        self.test_edges[edge_type][type_idx] = test_edges
        self.test_edges_false[edge_type][type_idx] = np.array(test_edges_false)

    def _ismember(self, a, b):
        a = np.array(a)
        b = np.array(b)
        rows_close = np.all(a - b == 0, axis=1)  #Subtracts the pair a(idx_i, idx_j) from each element of b, if equal then
        return np.any(rows_close)                #Function will return true if found a coordinates

    # Symteric (D^-0.5 . (A + I) . D^-0.5) and Assymteric (Dr ^ -0.5 . A . Dc ^ -0.5) normalization
    # For A(mxm) -> adj_ = A + I(mxm), R(mx1) = Sum of rows of adj_, M_inv = D(mxm) (diaginals are roots of R),
    # Adj_Normalized = (adj(mxm).D(mxm))'.D(mxm)
    #For A(mxn) -> R(mx1) = sum of rows of A, C(nx1) = sum of cols of A, RD(mxm) -> Diagonals(non-null) are roots of R,
    # CD(nxn) -> Diagonals(non-null) are roots of C
    # Adj_normalized = RD(mxm).A(mxn).CD(nxn)
    def preprocess_graph(self, adj):
        adj = sp.coo_matrix(adj)
        if adj.shape[0] == adj.shape[1]:
            adj_ = adj + sp.eye(adj.shape[0])
            rowsum = np.array(adj_.sum(1))
            degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
            adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
        else:
            rowsum = np.array(adj.sum(1))
            colsum = np.array(adj.sum(0))
            rowdegree_mat_inv = sp.diags(np.nan_to_num(np.power(rowsum, -0.5)).flatten())
            coldegree_mat_inv = sp.diags(np.nan_to_num(np.power(colsum, -0.5)).flatten())
            adj_normalized = rowdegree_mat_inv.dot(adj).dot(coldegree_mat_inv).tocoo()
        return sparse_to_tuple(adj_normalized)

    def update_feed_dict(self, feed_dict, dropout, placeholders):
        # construct feed dictionary
        feed_dict.update({
            placeholders['adj_mats_%d,%d,%d' % (i,j,k)]: self.adj_train[i,j][k]
            for i, j in self.edge_types for k in range(self.edge_types[i,j])})
        feed_dict.update({placeholders['feat_%d' % i]: self.feat[i] for i, _ in self.edge_types})
        feed_dict.update({placeholders['dropout']: dropout})
        return feed_dict

    def batch_feed_dict(self, batch_edges, batch_edge_type, placeholders):
        feed_dict = dict()
        feed_dict.update({placeholders['batch']: batch_edges})   # List of [x,y] coordinates
        feed_dict.update({placeholders['batch_edge_type_idx']: batch_edge_type}) # Type of edge, {0: (0,0,0), 1: (0,0,1) ..}
        feed_dict.update({placeholders['batch_row_edge_type']: self.idx2edge_type[batch_edge_type][0]}) # From Edge
        feed_dict.update({placeholders['batch_col_edge_type']: self.idx2edge_type[batch_edge_type][1]}) # To Edge
        return feed_dict


    def next_minibatch_feed_dict(self, placeholders):
        """Select a random edge type and a batch of edges of the same type"""

        self.current_edge_type_idx = self.iter%len(minibatch.idx2edge_type)
        print("Current Iteration: ", self.iter, "Current edge:", self.current_edge_type_idx)
        self.iter += 1

        if(len(current_epoch_loss[self.current_edge_type_idx]) > 0 and self.batch_num[self.current_edge_type_idx] == 0): #New Epoch starting, log and average losses
            epoch_losses_history[self.current_edge_type_idx].append(np.mean(current_epoch_loss[self.current_edge_type_idx]))
            if len(epoch_losses_history[self.current_edge_type_idx]) > 15:
                epoch_losses_history[self.current_edge_type_idx].pop(0)
            current_epoch_loss[self.current_edge_type_idx] = []


        i, j, k = self.idx2edge_type[self.current_edge_type_idx]
        print("Batch Number: ", self.batch_num[self.current_edge_type_idx])
        if self.batch_num[self.current_edge_type_idx] * self.batch_size \
                   > len(self.train_edges[i,j][k]) - self.batch_size + 1:   #Less than batch size elements left
            batch_edges = self.train_edges[i,j][k][self.batch_num[self.current_edge_type_idx] * self.batch_size:]
            #print("Extracting train_edges[", str(i), ",",str(j),"][",self.batch_num[self.current_edge_type_idx] * self.batch_size,":]")
            if(batch_edges.shape[0] < 512):
                batch_edges = np.concatenate([batch_edges, self.train_edges[i,j][k][0:self.batch_size - batch_edges.shape[0]]], axis = 0)
                #print("Extracting train_edges[", str(i), ",",str(j),"][0:",self.batch_size - batch_edges.shape[0],"]")
            else:
                #print("Too many elements added, taking only first ", self.batch_size, " elements")
                batch_edges = batch_edges[0:self.batch_size]
            self.batch_num[self.current_edge_type_idx] = 0
            assert batch_edges.shape == (self.batch_size,2)
            return self.batch_feed_dict(batch_edges, self.current_edge_type_idx, placeholders)



        start = self.batch_num[self.current_edge_type_idx] * self.batch_size
        self.batch_num[self.current_edge_type_idx] += 1
        #print("Extracting train_edges[", str(i), ",",str(j),"][", str(k),"][", start, ":", start + self.batch_size,"]")
        batch_edges = self.train_edges[i,j][k][start: start + self.batch_size]
        assert batch_edges.shape == (self.batch_size,2)
        return self.batch_feed_dict(batch_edges, self.current_edge_type_idx, placeholders)


In [ ]:
print("Create minibatch iterator")
minibatch = EdgeMinibatchIterator(
        adj_mats=adj_mats_orig,  #Check cell 26
        feat=feat,              #{ 0: [12331, 17480 feature matrix], 1: [3215, 16592 feature matrix] }
        edge_types=edge_types,   #{(0, 0): 2, (0, 1): 1, (1, 0): 1, (1, 1): 2}
        batch_size=batch_size,      #512
        val_test_size=val_test_size  #0.1
    )

In [ ]:
indices, values, shape = feat[1]

In [ ]:
indices

##  Defining The Model

In [ ]:
def weight_variable_glorot(input_dim, output_dim, name=""):
    """Create a weight variable with Glorot & Bengio (AISTATS 2010)
    initialization.
    """
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = tf.compat.v1.random_uniform([input_dim, output_dim], minval=-init_range,
                                maxval=init_range, dtype=tf.float32)
    return tf.Variable(initial, name=name)

In [ ]:
# global unique layer ID dictionary for layer name assignment
_LAYER_UIDS = {}


def get_layer_uid(layer_name=''):
    """Helper function, assigns unique layer IDs
    """
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]


def dropout_sparse(x, keep_prob, num_nonzero_elems):
    """Dropout for sparse tensors. Currently fails for very large sparse tensors (>1M elements) # TODO: CHANGE THIS
    """
    noise_shape = [num_nonzero_elems]
    random_tensor = keep_prob         # 1 - dropout
    random_tensor += tf.compat.v1.random_uniform(noise_shape)     #Add 1 - dropout to random matrix of Nx1, N is input
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)    # If > 1, keep or else drop
    pre_out = tf.compat.v1.sparse_retain(x, dropout_mask)
    return pre_out * (1./keep_prob)                             # The Remaining elements get multiplied, dropout concept


class MultiLayer(object):
    """Base layer class. Defines basic API for all layer objects.

    # Properties
        name: String, defines the variable scope of the layer.

    # Methods
        _call(inputs): Defines computation graph of layer
            (i.e. takes input, returns output)
        __call__(inputs): Wrapper for _call()
    """
    def __init__(self, edge_type=(), num_types=-1, **kwargs):
        self.edge_type = edge_type
        self.num_types = num_types
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}
        logging = kwargs.get('logging', False)
        self.logging = logging
        self.issparse = False

    def _call(self, inputs):
        return inputs

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            outputs = self._call(inputs)
            return outputs


class GraphConvolutionSparseMulti(MultiLayer):
    """Graph convolution layer for sparse inputs."""
    def __init__(self, input_dim, output_dim, adj_mats,
                 nonzero_feat, dropout=0., act=tf.nn.relu, **kwargs):
        super(GraphConvolutionSparseMulti, self).__init__(**kwargs) #input_dim = num_feat:  {0: 17480, 1: 16592}
        self.dropout = dropout
        self.adj_mats = adj_mats
        self.act = act
        self.issparse = True
        self.nonzero_feat = nonzero_feat  #nonzero_feat:  {0: 12331, 1: 3215}
        with tf.compat.v1.variable_scope('%s_vars' % self.name):      #weights_i are glorot variables: input_dimxoutput_dim matrix
            for k in range(self.num_types):
                print("Initializing weights: weights_%d with shape: " %k, "And edge type: ", self.edge_type,"num_types:", self.num_types)
                print(str(input_dim[self.edge_type[1]]) + " " + str(output_dim))
                self.vars['weights_%d' % k] = weight_variable_glorot(
                    input_dim[self.edge_type[1]], output_dim, name='weights_%d' % k) #input_dim:  {0: 17480, 1: 16592}
                                                                                    #output_dim: hidden_1 units = 64

    def _call(self, inputs):
        outputs = []
        for k in range(self.num_types):
            x = dropout_sparse(inputs, 1-self.dropout, self.nonzero_feat[self.edge_type[1]]) # x is shape 12331(3215)
            x = tf.compat.v1.sparse_tensor_dense_matmul(x, self.vars['weights_%d' % k])       # x 12331 * []
            x = tf.compat.v1.sparse_tensor_dense_matmul(self.adj_mats[self.edge_type][k], x)
            outputs.append(self.act(x))
        outputs = tf.add_n(outputs)
        outputs = tf.nn.l2_normalize(outputs, dim=1)
        return outputs


class GraphConvolutionMulti(MultiLayer):
    """Basic graph convolution layer for undirected graph without edge labels."""
    def __init__(self, input_dim, output_dim, adj_mats, dropout=0., act=tf.nn.relu, **kwargs):
        super(GraphConvolutionMulti, self).__init__(**kwargs)
        self.adj_mats = adj_mats
        self.dropout = dropout
        self.act = act
        with tf.compat.v1.variable_scope('%s_vars' % self.name):
            for k in range(self.num_types):
                self.vars['weights_%d' % k] = weight_variable_glorot(
                    input_dim, output_dim, name='weights_%d' % k)

    def _call(self, inputs):
        outputs = []
        for k in range(self.num_types):
            x = tf.nn.dropout(inputs, 1-self.dropout)
            x = tf.matmul(x, self.vars['weights_%d' % k])
            x = tf.compat.v1.sparse_tensor_dense_matmul(self.adj_mats[self.edge_type][k], x)
            outputs.append(self.act(x))
        outputs = tf.add_n(outputs)
        outputs = tf.nn.l2_normalize(outputs, dim=1)
        return outputs




class InnerProductDecoder(MultiLayer):
    """Decoder model layer for link prediction."""
    def __init__(self, input_dim, dropout=0., act=tf.nn.sigmoid, **kwargs):
        super(InnerProductDecoder, self).__init__(**kwargs)
        self.dropout = dropout
        self.act = act

    def _call(self, inputs):
        i, j = self.edge_type
        outputs = []
        for k in range(self.num_types):
            inputs_row = tf.nn.dropout(inputs[i], 1-self.dropout)
            inputs_col = tf.nn.dropout(inputs[j], 1-self.dropout)
            rec = tf.matmul(inputs_row, tf.transpose(inputs_col))
            outputs.append(self.act(rec))
        return outputs


# Decagon Model

In [ ]:
from collections import defaultdict

class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg

        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', True)
        self.logging = logging

        self.vars = {}

    def _build(self):
        raise NotImplementedError

    def build(self):
        """ Wrapper for _build() """
        with tf.compat.v1.variable_scope(self.name):
            self._build()
        variables = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

    def fit(self):
        pass

    def predict(self):
        pass


class DecagonModel(Model):
    def __init__(self, placeholders, num_feat, nonzero_feat, edge_types, decoders, **kwargs):
        super(DecagonModel, self).__init__(**kwargs)
        self.edge_types = edge_types    #{(0, 0): 2, (0, 1): 1, (1, 0): 1, (1, 1): 2}
        self.num_edge_types = sum(self.edge_types.values())      #6
        self.num_obj_types = max([i for i, _ in self.edge_types]) + 1       #2
        self.decoders = decoders #{(0, 0): 'innerproduct', (0, 1): 'innerproduct', (1, 0): 'innerproduct', (1, 1): 'innerproduct'}
        self.inputs = {i: placeholders['feat_%d' % i] for i, _ in self.edge_types} #sparse tensors with n*indices, n*values, shape
        self.input_dim = num_feat  #num_feat:  {0: 17480, 1: 16592}
        self.nonzero_feat = nonzero_feat #nonzero_feat:  {0: 12331, 1: 3215}
        self.placeholders = placeholders
        self.dropout = placeholders['dropout']  #default 0
        self.adj_mats = {et: [
            placeholders['adj_mats_%d,%d,%d' % (et[0], et[1], k)] for k in range(n)]
            for et, n in self.edge_types.items()}
        self.build()

    def _build(self):
        self.hidden1 = defaultdict(list)  #Create empty list when trying to access key not there
        for i, j in self.edge_types:
            self.hidden1[i].append(GraphConvolutionSparseMulti(
                input_dim=self.input_dim, output_dim=hidden1_units,
                edge_type=(i,j), num_types=self.edge_types[i,j],
                adj_mats=self.adj_mats, nonzero_feat=self.nonzero_feat,
                act=lambda x: x, dropout=self.dropout,       # no activation ???
                logging=self.logging)(self.inputs[j]))      # Logging is only true or false
            # self.hidden1[i].append(GraphConvolutionMulti(
            #     input_dim=self.input_dim, output_dim=hidden1_units,
            #     edge_type=(i,j), num_types=self.edge_types[i,j],
            #     adj_mats=self.adj_mats, act=lambda x: x,
            #     dropout=self.dropout,logging=self.logging)(self.inputs[j]))

        for i, hid1 in self.hidden1.items():
            self.hidden1[i] = tf.nn.relu(tf.add_n(hid1))

        self.embeddings_reltyp = defaultdict(list)
        for i, j in self.edge_types:
            self.embeddings_reltyp[i].append(GraphConvolutionMulti(
                input_dim=hidden1_units, output_dim=hidden2_units,
                edge_type=(i,j), num_types=self.edge_types[i,j],
                adj_mats=self.adj_mats, act=lambda x: x,
                dropout=self.dropout, logging=self.logging)(self.hidden1[j]))

        self.embeddings = [None] * self.num_obj_types
        for i, embeds in self.embeddings_reltyp.items():
            # self.embeddings[i] = tf.nn.relu(tf.add_n(embeds))
            self.embeddings[i] = tf.add_n(embeds)

        self.edge_type2decoder = {}
        for i, j in self.edge_types:
            decoder = self.decoders[i, j]
            if decoder == 'innerproduct':
                self.edge_type2decoder[i, j] = InnerProductDecoder(
                    input_dim=hidden2_units, logging=self.logging,
                    edge_type=(i, j), num_types=self.edge_types[i, j],
                    act=lambda x: x, dropout=self.dropout)
            else:
                raise ValueError('Unknown decoder type')

        self.latent_inters = []
        self.latent_varies = []
        for edge_type in self.edge_types:
            decoder = self.decoders[edge_type]
            for k in range(self.edge_types[edge_type]):
                if decoder == 'innerproduct':
                    glb = tf.eye(hidden2_units, hidden2_units)
                    loc = tf.eye(hidden2_units, hidden2_units)
                else:
                    raise ValueError('Unknown decoder type')

                self.latent_inters.append(glb)
                self.latent_varies.append(loc)


In [ ]:
print("Create model")
model = DecagonModel(
        placeholders=placeholders,
        num_feat=num_feat,
        nonzero_feat=nonzero_feat,
        edge_types=edge_types,
        decoders=edge_type2decoder,
)


# Decagon Optimizer

In [ ]:
negative_sampler_index_variable = 0

In [ ]:
class DecagonOptimizer(object):
    def __init__(self, embeddings, latent_inters, latent_varies,
                 degrees, edge_types, edge_type2dim, placeholders,
                 margin=0.1, neg_sample_weights=1., batch_size=100):
        self.embeddings= embeddings
        self.latent_inters = latent_inters
        self.latent_varies = latent_varies
        self.edge_types = edge_types
        self.degrees = degrees
        self.edge_type2dim = edge_type2dim           #(0, 0): [(12331, 12331), (12331, 12331)] ...
        self.obj_type2n = {i: self.edge_type2dim[i,j][0][0] for i, j in self.edge_types}  #{0: 12331, 1: 3215}
        self.margin = margin
        self.neg_sample_weights = neg_sample_weights
        self.batch_size = batch_size

        self.inputs = placeholders['batch']
        self.batch_edge_type_idx = placeholders['batch_edge_type_idx']
        self.batch_row_edge_type = placeholders['batch_row_edge_type']
        self.batch_col_edge_type = placeholders['batch_col_edge_type']


        self.row_inputs = tf.squeeze(gather_cols(self.inputs, [0]))
        self.col_inputs = tf.squeeze(gather_cols(self.inputs, [1]))

        obj_type_n = [self.obj_type2n[i] for i in range(len(self.embeddings))]
        self.obj_type_lookup_start = tf.cumsum([0] + obj_type_n[:-1])    #0, 12331
        self.obj_type_lookup_end = tf.cumsum(obj_type_n)                 #12331, 12331 + 3215

        labels = tf.reshape(tf.cast(self.row_inputs, dtype=tf.int64), [self.batch_size, 1])

        self.neg_samples_list = []
        print_ops = []
        for i, j in self.edge_types:
            for k in range(self.edge_types[i,j]):
                neg_samples, _, _ = tf.nn.fixed_unigram_candidate_sampler(
                        true_classes=labels,
                        num_true=1,
                        num_sampled=self.batch_size,
                        unique=False,
                        range_max=len(self.degrees[i][k]),
                        distortion=0.75,
                        unigrams=self.degrees[i][k].tolist())
                self.neg_samples_list.append(neg_samples)

        print("Negative Samples List Created: ", self.neg_samples_list)

        print_op = tf.print("Index of Negative Sampler: ", negative_sampler_index_variable, "Negative Samples: ", self.neg_samples_list[negative_sampler_index_variable])
        with tf.control_dependencies([print_op]):
          self.neg_samples = self.neg_samples_list[negative_sampler_index_variable]
        #self.neg_samples_rows = tf.squeeze(gather_cols(self.neg_samples, [0]))

        self.preds = self.batch_predict(self.row_inputs, self.col_inputs)   # Diagonol has the product of the row-col embeddings
        self.outputs = tf.compat.v1.diag_part(self.preds)
        self.outputs = tf.reshape(self.outputs, [-1])

        self.neg_preds = self.batch_predict(self.neg_samples, self.col_inputs)
        self.neg_outputs = tf.compat.v1.diag_part(self.neg_preds)
        self.neg_outputs = tf.reshape(self.neg_outputs, [-1])

        self.predict()

        self._build()

    def batch_predict(self, row_inputs, col_inputs):
        concatenated = tf.concat(self.embeddings, 0)  # 12331x32 + 3215x32 together along 0 axis

        ind_start = tf.gather(self.obj_type_lookup_start, self.batch_row_edge_type)
        ind_end = tf.gather(self.obj_type_lookup_end, self.batch_row_edge_type)
        indices = tf.range(ind_start, ind_end)
        row_embeds = tf.gather(concatenated, indices)            # Only gather embeddings for the partcular type, 1st 12331, or next 3215
        row_embeds = tf.gather(row_embeds, row_inputs)         # Takes the 512 embeddings in current batch

        ind_start = tf.gather(self.obj_type_lookup_start, self.batch_col_edge_type) # Gather embeddings for Col type
        ind_end = tf.gather(self.obj_type_lookup_end, self.batch_col_edge_type)
        indices = tf.range(ind_start, ind_end)
        col_embeds = tf.gather(concatenated, indices)
        col_embeds = tf.gather(col_embeds, col_inputs)        # Takes the 512 embeddings in current batch

        latent_inter = tf.gather(self.latent_inters, self.batch_edge_type_idx)  # Identity matrix 32x32
        latent_var = tf.gather(self.latent_varies, self.batch_edge_type_idx)     # Identity matrix 32x32

        product1 = tf.matmul(row_embeds, latent_var)   # No effect in Innerproduct for all 3 products
        product2 = tf.matmul(product1, latent_inter)
        product3 = tf.matmul(product2, latent_var)
        preds = tf.matmul(product3, tf.transpose(col_embeds))  #512x32 * 32*512
        return preds          # Predition is simply product of two embeddings

    def predict(self):
        concatenated = tf.concat(self.embeddings, 0)

        ind_start = tf.gather(self.obj_type_lookup_start, self.batch_row_edge_type)
        ind_end = tf.gather(self.obj_type_lookup_end, self.batch_row_edge_type)
        indices = tf.range(ind_start, ind_end)
        row_embeds = tf.gather(concatenated, indices)

        ind_start = tf.gather(self.obj_type_lookup_start, self.batch_col_edge_type)
        ind_end = tf.gather(self.obj_type_lookup_end, self.batch_col_edge_type)
        indices = tf.range(ind_start, ind_end)
        col_embeds = tf.gather(concatenated, indices)

        latent_inter = tf.gather(self.latent_inters, self.batch_edge_type_idx)
        latent_var = tf.gather(self.latent_varies, self.batch_edge_type_idx)

        product1 = tf.matmul(row_embeds, latent_var)
        product2 = tf.matmul(product1, latent_inter)
        product3 = tf.matmul(product2, latent_var)
        self.predictions = tf.matmul(product3, tf.transpose(col_embeds))    # Predition is simply product of two embeddings

    def _build(self):
        self.cost = self._hinge_loss(self.outputs, self.neg_outputs)
        #self.cost = self._xent_loss(self.outputs, self.neg_outputs)
        self.optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate = learning_rate)
        update_ops = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS)
        #grads_and_vars = optim.compute_gradients(total_loss, var_list=train_vars)
        #train_op = optim.apply_gradients(grads_and_vars)
        self.train_vars = [var for var in tf.compat.v1.trainable_variables()]
        print("Trainable Variables: ", self.train_vars)
        with tf.control_dependencies(update_ops):
           self.opt_op = self.optimizer.minimize(self.cost,var_list=self.train_vars)

    def _hinge_loss(self, aff, neg_aff):
        """Maximum-margin optimization using the hinge loss."""
        diff = tf.nn.relu(tf.subtract(neg_aff, tf.expand_dims(aff, 0) - self.margin), name='diff')
        loss = tf.reduce_sum(diff)
        return loss

    def _xent_loss(self, aff, neg_aff):
        """Cross-entropy optimization."""
        true_xent = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(aff), logits=aff)
        negative_xent = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(neg_aff), logits=neg_aff)
        loss = tf.reduce_sum(true_xent) + self.neg_sample_weights * tf.reduce_sum(negative_xent)
        return loss


def gather_cols(params, indices, name=None):
    """Gather columns of a 2D tensor.

    Args:
        params: A 2D tensor.
        indices: A 1D tensor. Must be one of the following types: ``int32``, ``int64``.
        name: A name for the operation (optional).

    Returns:
        A 2D Tensor. Has the same type as ``params``.
    """
    with tf.compat.v1.op_scope([params, indices], name, "gather_cols") as scope:
        # Check input
        params = tf.convert_to_tensor(params, name="params")
        indices = tf.convert_to_tensor(indices, name="indices")
        try:
            params.get_shape().assert_has_rank(2)
        except ValueError:
            raise ValueError('\'params\' must be 2D.')
        try:
            indices.get_shape().assert_has_rank(1)
        except ValueError:
            raise ValueError('\'params\' must be 1D.')

        # Define op
        p_shape = tf.shape(params)
        p_flat = tf.reshape(params, [-1])
        i_flat = tf.reshape(tf.reshape(tf.range(0, p_shape[0]) * p_shape[1],
                                       [-1, 1]) + indices, [-1])
        return tf.reshape(
            tf.gather(p_flat, i_flat), [p_shape[0], -1])

In [ ]:
print("Create optimizer")
with tf.name_scope('optimizer'):
        opt = DecagonOptimizer(
            embeddings=model.embeddings,
            latent_inters=model.latent_inters,
            latent_varies=model.latent_varies,
            degrees=degrees,
            edge_types=edge_types,
            edge_type2dim=edge_type2dim,
            placeholders=placeholders,
            batch_size=batch_size,
            margin=max_margin
        )

In [ ]:
def bedroc_score(y_true, y_pred, decreasing=True, alpha=20.0):

    """BEDROC metric implemented according to Truchon and Bayley.

    The Boltzmann Enhanced Descrimination of the Receiver Operator
    Characteristic (BEDROC) score is a modification of the Receiver Operator
    Characteristic (ROC) score that allows for a factor of *early recognition*.

    References:
        The original paper by Truchon et al. is located at `10.1021/ci600426e
        <http://dx.doi.org/10.1021/ci600426e>`_.

    Args:
        y_true (array_like):
            Binary class labels. 1 for positive class, 0 otherwise.
        y_pred (array_like):
            Prediction values.
        decreasing (bool):
            True if high values of ``y_pred`` correlates to positive class.
        alpha (float):
            Early recognition parameter.

    Returns:
        float:
            Value in interval [0, 1] indicating degree to which the predictive
            technique employed detects (early) the positive class.
     """

    assert len(y_true) == len(y_pred), \
        'The number of scores must be equal to the number of labels'

    big_n = len(y_true)
    n = sum(y_true == 1)

    if decreasing:
        order = np.argsort(-y_pred)
    else:
        order = np.argsort(y_pred)

    m_rank = (y_true[order] == 1).nonzero()[0]

    s = np.sum(np.exp(-alpha * m_rank / big_n))

    r_a = n / big_n

    rand_sum = r_a * (1 - np.exp(-alpha))/(np.exp(alpha/big_n) - 1)

    fac = r_a * np.sinh(alpha / 2) / (np.cosh(alpha / 2) -
                                      np.cosh(alpha/2 - alpha * r_a))

    cte = 1 / (1 - np.exp(alpha * (1 - r_a)))

    return s * fac / rand_sum + cte



In [ ]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average precision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def ark(actual, predicted, k=10):

    if len(predicted)>k:
        predicted = predicted[:k]

    num_actual = len(actual)

    num_hits = 0.0
    if len(actual)==0:
        return 0

    for i, p in enumerate(actual):
        if p in predicted:
            num_hits += 1.0


    return num_hits / len(actual)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average precision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a, p in zip(actual, predicted)])


In [ ]:
def get_accuracy_scores(edges_pos, edges_neg, edge_type, name=None):

    negative_sampler_index_variable = 3
    feed_dict.update({placeholders['dropout']: 0})
    feed_dict.update({placeholders['batch_edge_type_idx']: minibatch.edge_type2idx[edge_type]})
    feed_dict.update({placeholders['batch_row_edge_type']: edge_type[0]})
    feed_dict.update({placeholders['batch_col_edge_type']: edge_type[1]})
    fetches = {
            'predictions': opt.predictions,
    }

    res = sess.run(fetches, feed_dict=feed_dict)
    rec = res['predictions']


    def sigmoid(x):
        return 1. / (1 + np.exp(-x))



    preds = []
    actual = []
    predicted = []
    edge_ind = 0
    for u, v in edges_pos[edge_type[:2]][edge_type[2]]:
        score = sigmoid(rec[u, v])
        preds.append(score)

        assert adj_mats_orig[edge_type[:2]][edge_type[2]][u,v] == 1, 'Problem 1'

        actual.append(edge_ind)
        predicted.append((score, edge_ind))
        edge_ind += 1

    preds_neg = []
    for u, v in edges_neg[edge_type[:2]][edge_type[2]]:
        score = sigmoid(rec[u, v])
        preds_neg.append(score)
        assert adj_mats_orig[edge_type[:2]][edge_type[2]][u,v] == 0, 'Problem 0'

        predicted.append((score, edge_ind))
        edge_ind += 1

    all_edge_idx = list(range(minibatch.train_edges[edge_type[:2]][edge_type[2]].shape[0]))
    np.random.shuffle(all_edge_idx)
    train_edge_idx = all_edge_idx[:100]
    train_preds = []
    for u, v in minibatch.train_edges[edge_type[:2]][edge_type[2]][train_edge_idx]:
        score = sigmoid(rec[u, v])
        train_preds.append(score)
        assert adj_mats_orig[edge_type[:2]][edge_type[2]][u,v] == 1, 'Problem 0'

        predicted.append((score, edge_ind))
        edge_ind += 1

    preds_mean = np.mean(preds)
    neg_preds_mean = np.mean(preds_neg)
    train_mean = np.mean(train_preds)
    preds_all = np.hstack([preds, preds_neg])
    preds_all = np.nan_to_num(preds_all)
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    predicted = list(zip(*sorted(predicted, reverse=True, key=itemgetter(0))))[1]

    roc_sc = metrics.roc_auc_score(labels_all, preds_all)
    aupr_sc = metrics.average_precision_score(labels_all, preds_all)
    apk_sc = apk(actual, predicted, k=200)
    bedroc_sc = bedroc_score(labels_all, preds_all)
    if name!=None:
        with open(name, 'wb') as f:
            pickle.dump([labels_all, preds_all], f)
    return roc_sc, aupr_sc, apk_sc, bedroc_sc, preds_mean, neg_preds_mean, train_mean

In [ ]:
def get_prediction(edges_pos, edges_neg, edge_type):
    feed_dict.update({placeholders['dropout']: 0})
    feed_dict.update({placeholders['batch_edge_type_idx']: minibatch.edge_type2idx[edge_type]})
    feed_dict.update({placeholders['batch_row_edge_type']: edge_type[0]})
    feed_dict.update({placeholders['batch_col_edge_type']: edge_type[1]})
    rec = sess.run(opt.predictions, feed_dict=feed_dict)

    return 1. / (1 + np.exp(-rec))

In [ ]:
tf.compat.v1.global_variables()

In [ ]:
print("Initialize session")
checkpoint_dir = '/content/drive/MyDrive/PGCN/model/checkpoints/'
saver = tf.compat.v1.train.Saver([var for var in tf.compat.v1.global_variables()], max_to_keep=10)
sess = tf.compat.v1.Session()

feed_dict = {}

checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if checkpoint is not None:
  print("Restoring checkpoint from {}".format(checkpoint))
  saver.restore(sess, checkpoint)
else:
  print("Initializing variables")
  sess.run(tf.compat.v1.global_variables_initializer())
#grads_and_vars = opt.optimizer.compute_gradients(opt.cost, var_list=train_vars)
#train_op = opt.optimizer.apply_gradients(grads_and_vars)


In [ ]:
try:
  epoch_losses_history = np.ndarray.tolist(np.load(os.path.join(checkpoint_dir, 'epoch_losses.npy'), allow_pickle=True))
  print("Previous epoch losses:", epoch_losses_history)
except:
  epoch_losses_history = defaultdict(list)

try:
  minibatch.batch_num = np.ndarray.tolist(np.load(os.path.join(checkpoint_dir, 'batch_numbers.npy'), allow_pickle=True))
  print("Previous batch Numbers:", minibatch.batch_num)
except:
  minibatch.batch_num = [0]*minibatch.num_edge_types

In [ ]:
from IPython.display import clear_output
for step in range(0, 30000):
    negative_sampler_index_variable = step%6
    feed_dict = minibatch.next_minibatch_feed_dict(placeholders=placeholders)
    feed_dict = minibatch.update_feed_dict(
        feed_dict=feed_dict,
        dropout=dropout,
        placeholders=placeholders)


    fetches = {
          'loss': opt.cost,
          'train': opt.opt_op
        }


    results = sess.run(fetches, feed_dict = feed_dict)
    #print("Loss:", results['loss'])
    current_epoch_loss[minibatch.current_edge_type_idx].append(results['loss'])

    if(step%100 == 1):
        clear_output()
        saver.save(sess, os.path.join(checkpoint_dir, 'model.latest'))
        np.save(os.path.join(checkpoint_dir, 'epoch_losses.npy'), epoch_losses_history)
        np.save(os.path.join(checkpoint_dir, 'batch_numbers.npy'), minibatch.batch_num)
        for i in [3]:
            edge_type = minibatch.idx2edge_type[i]
            roc_score, auprc_score, apk_score, bedroc, val_preds, val_negs, train_preds = get_accuracy_scores(
                minibatch.val_edges, minibatch.val_edges_false, minibatch.idx2edge_type[i])
            print("Edge type=", "[%02d, %02d, %02d]" % minibatch.idx2edge_type[i])
            print("Edge type:", "%04d" % 3, "Val Preds Avg", "{:.5f}".format(val_preds))
            print("Edge type:", "%04d" % 3, "Val Neg Avg", "{:.5f}".format(val_negs))
            print("Edge type:", "%04d" % 3, "Val AUROC score", "{:.5f}".format(roc_score))
            print("Edge type:", "%04d" % 3, "Val AUPRC score", "{:.5f}".format(auprc_score))
            print("Edge type:", "%04d" % 3, "Val AP@k score", "{:.5f}".format(apk_score))
            print("Edge type:", "%04d" % 3, "Val BEDROC score", "{:.5f}".format(bedroc))
            print("All Edge Average Losses", epoch_losses_history)
            print()



prediction = get_prediction(minibatch.test_edges, minibatch.test_edges_false,
    	minibatch.idx2edge_type[3])

print('Saving result...')
np.save(os.path.join(checkpoint_dir,'prediction.npy'), prediction)

In [ ]:
minibatch.train_edges[0,0][0].size